In [1]:
import warnings #Ignore warnings
warnings.filterwarnings('ignore')
import folium as folium #Use Folium library for map
import pandas as pd #Pandas
import numpy as np #Import plotnine
from plotnine import *
import folium
import branca
import matplotlib.pyplot as plt #Import Matplotlib

In [2]:
shop_2018 = pd.read_csv ('C:/Users/User/Downloads/traffic2017.csv', encoding = 'cp949')

In [6]:
shop_2018['dateslice'] = shop_2018['발생년월일시']
shop_2018['dateslice'] = shop_2018['dateslice'].astype(str).str[:-2].astype(np.int64)
shop_2018['date'] = pd.to_datetime(shop_2018['dateslice'], format='%Y%m%d')
shop_2018["mins"] = shop_2018.발생분.map("{:02}".format)
shop_2018['timeslice'] = shop_2018['발생년월일시'].astype(str).str[-2:].astype(np.int64)
shop_2018['var3'] = shop_2018.dateslice.astype(str) + shop_2018.mins.astype(str)
shop_2018['daters'] = pd.to_datetime(shop_2018['var3'], format='%Y%m%d%H%M')
shop_2018['month'] = shop_2018['dateslice'].astype(str).str[:-2].astype(np.int64)

In [7]:
shop_2018['month'] = shop_2018['month'].astype(str).str[-2:].astype(np.int64)
df = shop_2018.reset_index()
df.index.name = 'accident'
df = df.reset_index()

In [8]:
def fancy_html(row):
    i = row
    
    Number_of_Vehicles = df['사상자수'].iloc[i]                             
    Number_of_Casualties = df['사망자수'].iloc[i]                           
    Date = df['daters'].iloc[i]
    Time = df['daters'].iloc[i]                                           
    Light_Conditions = df['주야'].iloc[i]                               
    Weather_Conditions = df['사고유형_대분류'].iloc[i]                             
    Road_Surface_Conditions = df['발생지시도'].iloc[i]
    
    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:0"; width="300px">{}</h4>""".format(Date) + """

</head>
    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">사상자수</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Number_of_Vehicles) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Casualties</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Number_of_Casualties) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Time</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Time) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">주야</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Light_Conditions) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">사고유형_대분류</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Weather_Conditions) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">발생지시도</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Road_Surface_Conditions) + """
</tr>
</tbody>
</table>
</html>
"""
    return html

In [9]:
geo_df2 = df.loc[0:500,:] 

In [12]:
location = geo_df2['위도'].mean(), geo_df2['경도'].mean()
m = folium.Map(location=location,zoom_start=15,min_zoom=5)

for i in range(0,len(geo_df2)):
    html = fancy_html(i)
    num_of_casualties = geo_df2['사망자수'].iloc[i]
    if num_of_casualties == 1:
       color = 'blue'
    elif num_of_casualties == 2:
       color = 'green'
    else:
       color = 'red'
    iframe = branca.element.IFrame(html=html,width=400,height=300)
    popup = folium.Popup(iframe,parse_html=True)
   
    folium.Marker([geo_df2['위도'].iloc[i],geo_df2['경도'].iloc[i]],
                  popup=popup,icon=folium.Icon(color=color, icon='info-sign')).add_to(m)

m